In [1]:
# Auto-reload internal modules
%load_ext autoreload
%autoreload 2

# External modules
import numpy as np

# Internal modules
from cosmotd.domain_walls import plot_domain_wall_simulation
from cosmotd.cosmic_strings import plot_cosmic_string_simulation
from cosmotd.charged_domain_walls import plot_charged_domain_wall_simulation
from cosmotd.pentavac import plot_pentavac_simulation
from cosmotd.companion_axion import plot_companion_axion_simulation
from cosmotd.plot import *
from cosmotd.parameters import *

In [2]:
# Size of simulation grid
N = 50
# Model parameters
beta = 1
eta_phi = 1
eta_sigma = np.sqrt(3) / 2
lam_phi = 2
lam_sigma = 2
charge_density = 0.1
# RNG seed
seed = 23213241
# Run time
run_time = None
# Plotter
plotter = MplNotebookPlotter

In [ ]:
# Domain walls simulation
plot_domain_wall_simulation(N, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, MplMultiPlotter, run_time, seed)

In [ ]:
# Cosmic strings simulation
plot_cosmic_string_simulation(N, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, plotter, run_time, seed)

In [ ]:
# Charged domain walls simulation
plot_charged_domain_wall_simulation(
    N,
    DX,
    DT,
    ALPHA_2D,
    beta,
    eta_phi,
    eta_sigma,
    lam_phi,
    lam_sigma,
    charge_density,
    RADIATION_ERA,
    MplNotebookPlotter,
    run_time,
    seed
)

In [ ]:
# Pentavac model simulation
epsilon = 0.1
plot_pentavac_simulation(N, N, DX, DT, ALPHA_2D, epsilon, RADIATION_ERA, plotter, run_time, seed)

In [6]:
# Companion axion
plot_companion_axion_simulation(N, DX, DT, ALPHA_2D, eta_phi, RADIATION_ERA, 5, MplMp4Plotter, run_time, seed)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 251/251 [05:34<00:00,  1.33s/it]
ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enabl